In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# setting parameters for papermill

consultant_name = 'nick'

In [3]:
from fsspec.implementations.local import LocalFileSystem
import re
import os
import sys
import dateutil.parser as parser
from ibots_db.schema import ConsultingReport
from ibots_db import update_all, load
from pathlib import Path

## Getting file paths to entries

In [4]:
fs = LocalFileSystem()
fparent = f'data/entries/{consultant_name}/'
entry_files = list(Path(fparent).glob('*.txt'))
assert len(entry_files) > 0, 'No files found'
print(f'Found {len(entry_files)} entries from {consultant_name}')


Found 55 entries from nick


## Transform to dictionary

In [5]:
fun_split = lambda s: re.split(r'(Type:|Scholar:|Topic:|Date:|Content:)', s)
fun_key_add = lambda d: {key: d.get(key, '') for key in ['topic', 'scholar', 'date', 'content', 'type']}

sessions = []

for entry_ind, entry_file in enumerate(entry_files):
    session = {'type':'', 'date':'', 'topic':'', 'content':'', 'scholar':''}
    print(f"Getting dictionary from {entry_file}")
    entry_text = open(entry_file, 'r', encoding='utf-8').read()
    session = {entry_splits[i].lower().strip(':'): entry_splits[i + 1].strip() for entry_splits in [fun_split(entry_text)] for i in range(1, len(entry_splits), 2)}
    session['date'] = parser.parse(session['date']).strftime('%Y-%m-%d')
    session = fun_key_add(session)
    session['consultant'] = consultant_name
    sessions.append(session)

Getting dictionary from data\entries\nick\000.txt
Getting dictionary from data\entries\nick\002.txt
Getting dictionary from data\entries\nick\003.txt
Getting dictionary from data\entries\nick\004.txt
Getting dictionary from data\entries\nick\005.txt
Getting dictionary from data\entries\nick\006.txt
Getting dictionary from data\entries\nick\007.txt
Getting dictionary from data\entries\nick\008.txt
Getting dictionary from data\entries\nick\009.txt
Getting dictionary from data\entries\nick\010.txt
Getting dictionary from data\entries\nick\011.txt
Getting dictionary from data\entries\nick\012.txt
Getting dictionary from data\entries\nick\013.txt
Getting dictionary from data\entries\nick\014.txt
Getting dictionary from data\entries\nick\015.txt
Getting dictionary from data\entries\nick\016.txt
Getting dictionary from data\entries\nick\017.txt
Getting dictionary from data\entries\nick\018.txt
Getting dictionary from data\entries\nick\019.txt
Getting dictionary from data\entries\nick\020.txt


## Validating entries

In [7]:
entries = [ConsultingReport(**entry) for entry in sessions]